## Notes
Questions:
* Train on all 3 rivers, or individual rivers? River transfer knowledge?
* Which giver better result for 5 day forcast: seq to seq or rolling predictions?

In [20]:
import pandas as pd
import numpy as np
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
#from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6
rcParams['axes.titlesize'] = 'xx-large'
rcParams['axes.titleweight'] = 'bold'
rcParams["legend.loc"] = 'upper left'
look = 15
lead = 1 # 3, 5, 7, 10

In [21]:
#1934-4-01  2018-07-09
Qx = pd.read_csv('../../data/streamflw_precipitation/X_Ganges.csv', index_col=1,header=0,parse_dates=True) #Brahmaputra #Meghna
print (Qx.head())
print("End Qx.head()")
#print (Qx.index[-220:])
idx = []
for i in np.arange(1-look,1,1):
    idx.append('Q_'+str(i))
Qx = Qx.loc[:,idx]
#print (Qx.head())
Qy = pd.read_csv('../../data/streamflw_precipitation/Y_Ganges.csv', index_col=1,header=0,parse_dates=True)
#print (Qy.head())
idy = []
for i in np.arange(lead,lead+10,1):
    idy.append('Q_'+str(i))
Qy = Qy.loc[:,idy]
#print (Qy.head())

Qy.head()

            Unnamed: 0  Q_-20  Q_-19  Q_-18  Q_-17  Q_-16  Q_-15  Q_-14  \
Date                                                                      
1985-05-01         120  739.0  728.0  717.0  754.0  821.0  819.0  824.0   
1985-05-02         121  728.0  717.0  754.0  821.0  819.0  824.0  814.0   
1985-05-03         122  717.0  754.0  821.0  819.0  824.0  814.0  815.0   
1985-05-04         123  754.0  821.0  819.0  824.0  814.0  815.0  803.0   
1985-05-05         124  821.0  819.0  824.0  814.0  815.0  803.0  793.0   

            Q_-13  Q_-12  ...     Q_-9   Q_-8   Q_-7   Q_-6   Q_-5   Q_-4  \
Date                      ...                                               
1985-05-01  814.0  815.0  ...    800.0  805.0  813.0  824.0  829.0  829.0   
1985-05-02  815.0  803.0  ...    805.0  813.0  824.0  829.0  829.0  852.0   
1985-05-03  803.0  793.0  ...    813.0  824.0  829.0  829.0  852.0  810.0   
1985-05-04  793.0  800.0  ...    824.0  829.0  829.0  852.0  810.0  756.0   
1985-05-05  

,Q_1,Q_2,Q_3,Q_4,Q_5,Q_6,Q_7,Q_8,Q_9,Q_10
Date,,,,,,,,,,
1985-05-01,830.0,903.0,934.0,952.0,958.0,968.0,918.0,783.0,838.0,922.0
1985-05-02,903.0,934.0,952.0,958.0,968.0,918.0,783.0,838.0,922.0,893.0
1985-05-03,934.0,952.0,958.0,968.0,918.0,783.0,838.0,922.0,893.0,874.0
1985-05-04,952.0,958.0,968.0,918.0,783.0,838.0,922.0,893.0,874.0,879.0
1985-05-05,958.0,968.0,918.0,783.0,838.0,922.0,893.0,874.0,879.0,861.0


## Data Processing
Reshaping and Normalization

TODO: 
Filtering:
1. Moving average (np.convolve, see 2nd answer https://stackoverflow.com/questions/13728392/moving-average-or-running-mean), could also add a convolution layer?
2. Local Regression http://www.statsmodels.org/stable/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html
3. Savitzky-Golay filtering (scipy)
4. Hamming Window Filtering (numpy and scipy)

## Unrolling Functions
Impliment functions to unroll data windows into 1d, and reroll into 2d windows

Having issues with indexing

In [22]:
# unroll data
# FIX INDEXING! REMEMBER, DATES REFER TO Q_0, parse strings to ints if necessary! or unroll from last digit
def unroll_x(df):
    bulk = df.iloc[:-1,-1]
    edgeCase = df.iloc[-1].T
    unrolled = edgeCase.append(bulk)
    return unrolled

def unroll_y(df):
    bulk = df.iloc[:-1,-1]
    edgeCase = df.iloc[-1].T
    unrolled = bulk.append(edgeCase)
    return unrolled  

print(Qy)
print(".............")
print(".............")
print(".............")
print(unroll_y(Qy))

                Q_1      Q_2      Q_3      Q_4      Q_5      Q_6      Q_7  \
Date                                                                        
1985-05-01    830.0    903.0    934.0    952.0    958.0    968.0    918.0   
1985-05-02    903.0    934.0    952.0    958.0    968.0    918.0    783.0   
1985-05-03    934.0    952.0    958.0    968.0    918.0    783.0    838.0   
1985-05-04    952.0    958.0    968.0    918.0    783.0    838.0    922.0   
1985-05-05    958.0    968.0    918.0    783.0    838.0    922.0    893.0   
1985-05-06    968.0    918.0    783.0    838.0    922.0    893.0    874.0   
1985-05-07    918.0    783.0    838.0    922.0    893.0    874.0    879.0   
1985-05-08    783.0    838.0    922.0    893.0    874.0    879.0    861.0   
1985-05-09    838.0    922.0    893.0    874.0    879.0    861.0    832.0   
1985-05-10    922.0    893.0    874.0    879.0    861.0    832.0    810.0   
1985-05-11    893.0    874.0    879.0    861.0    832.0    810.0    787.0   

In [44]:
#Can roll back up the original dataframes
# Needs work
def roll(df, original):
    temp = original
    temp = temp - original
    original_columns = len(temp.columns)
    total_rows = len(temp.index) - original_columns
    for i in range(total_rows):
        for j in range(original_columns):
            temp.iloc[i][j] = df.iloc[i+j]
    return temp
        
        
#testing
print(Qx.head(2))
print("............")
print("............")
unrollx = unroll_x(Qx)
print(unrollx.head(2))
print("............")
print("............")
rollx = roll(unrollx, Qx)
print(rollx.eq(Qx))
print("............")
print("............")
print("............")
print("............")
print(Qy.head(2))
print("............")
print("............")
unrolly = unroll_y(Qy)
print(unrolly.head(2))
print("............")
print("............")
rollx = roll(unrolly, Qy)
print(rollx.eq(Qy))
print("............")
print("............")
print("............")
print("............")

               Q_-14     Q_-13     Q_-12     Q_-11     Q_-10      Q_-9  \
Date                                                                     
1985-05-01  112121.0  112121.0  112121.0  112121.0  112121.0  112121.0   
1985-05-02  112121.0  112121.0  112121.0  112121.0  112121.0  112121.0   

                Q_-8      Q_-7      Q_-6      Q_-5      Q_-4      Q_-3  \
Date                                                                     
1985-05-01  112121.0  112121.0  112121.0  112121.0  112121.0  112121.0   
1985-05-02  112121.0  112121.0  112121.0  112121.0  112121.0  112121.0   

                Q_-2      Q_-1       Q_0  
Date                                      
1985-05-01  112121.0  112121.0  112121.0  
1985-05-02  112121.0  112121.0  112121.0  
............
............
Q_-14    112120.0
Q_-13    112120.0
dtype: float64
............
............
            Q_-14  Q_-13  Q_-12  Q_-11  Q_-10   Q_-9   Q_-8   Q_-7   Q_-6  \
Date                                                  

              Q_1    Q_2    Q_3    Q_4    Q_5    Q_6    Q_7    Q_8    Q_9  \
Date                                                                        
1985-05-01  False  False  False  False  False  False  False  False  False   
1985-05-02  False  False  False  False  False  False  False  False  False   
1985-05-03  False  False  False  False  False  False  False  False  False   
1985-05-04  False  False  False  False  False  False  False  False  False   
1985-05-05  False  False  False  False  False  False  False  False  False   
1985-05-06  False  False  False  False  False  False  False  False  False   
1985-05-07  False  False  False  False  False  False  False  False  False   
1985-05-08  False  False  False  False  False  False  False  False  False   
1985-05-09  False  False  False  False  False  False  False  False  False   
1985-05-10  False  False  False  False  False  False  False  False  False   
1985-05-11  False  False  False  False  False  False  False  False  False   

## Filtering

In [24]:
# Local Regression
from statsmodels.nonparametric import smoothers_lowess

http://www.statsmodels.org/stable/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html

In [26]:
# Savitzky-Golay and hamming
from scipy import signal

https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.signal.savgol_filter.html

https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.signal.hamming.html#scipy.signal.hamming

In [4]:
X = np.swapaxes(Qx.values[np.newaxis], 0, 1)
y = np.swapaxes(Qy.values[np.newaxis], 0, 1)

In [5]:
print(X.shape)
print(y.shape)

(4896, 1, 15)
(4896, 1, 10)


In [48]:
print(len(Qx.columns))

15


In [28]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Conv1D,Flatten

from tensorflow.keras import initializers, regularizers

from keras_pandas.Automater import Automater

## Params
Default params for networks

In [7]:
# Rolling window of X
rolling_window = len(Qx.columns)

# of epochs
numEpoch = 5

# Which day are we predicting?
y = Qy['Q_1']
y.head().squeeze()

Date
1985-05-01    830.0
1985-05-02    903.0
1985-05-03    934.0
1985-05-04    952.0
1985-05-05    958.0
Name: Q_1, dtype: float64

In [8]:
l2 = regularizers.l2
bias = initializers.glorot_uniform(seed = 0)
#kernal = 

## Dense

In [9]:
def initDense():
    # Initialize network as sequential
    model = Sequential()
    
    # Add reccurant layer
    model.add(Dense(10,
                        input_shape = (1, rolling_window),))
    
    # Add dense layer
    model.add(Dense(1,))
    
    #return model
    return model
    
initDense().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 10)             160       
_________________________________________________________________
dense_1 (Dense)              (None, 1, 10)             110       
Total params: 270
Trainable params: 270
Non-trainable params: 0
_________________________________________________________________


## RNN

In [10]:
def initRNN():
    # Initialize network as sequential
    model = Sequential()
        
    # Add reccurant layer
    model.add(SimpleRNN(10,
                        input_shape = (1, rolling_window),))
    
    # Syncronize with input shape    
    #model.add(Flatten())

    
    # Add dense layer
    model.add(Dense(1))
    
    #return model
    return model
    
initRNN().summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10)                260       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________


## LSTM

In [11]:
def initLSTM():
    # Initialize network as sequential
    model = Sequential()
    
    # Add reccurant LSTM layer
    model.add(LSTM(10,
                        input_shape = (1, rolling_window),))
    
    # Add dense layer
    model.add(Dense(1,))
    
    #return model
    return model
    
initLSTM().summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10)                1040      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 1,051
Trainable params: 1,051
Non-trainable params: 0
_________________________________________________________________


# CNN
Issues:
Input shape dimensionality

In [36]:
def initCNN():
    # Initialize network as sequential
    model = Sequential()
    
    # Add reccurant LSTM layer
    model.add(Conv1D(filters = 10, kernel_size = 2, input_shape = (1, rolling_window),))
    model.add(Flatten())
    # Add dense layer
    model.add(Dense(1,))
    
    #return model
    return model
    
initConv1D().summary() 

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'conv1d/conv1d/Conv2D' (op: 'Conv2D') with input shapes: [?,1,1,15], [1,2,15,10].

## Arima

In [13]:
from statsmodels.tsa.arima_model import ARIMA

In [14]:
QxV = Qx.values
print(QxV)
print(QxV.shape)
print(Qx)
print(Qx.shape)

[[  824.   814.   815. ...   810.   756.   751.]
 [  814.   815.   803. ...   756.   751.   830.]
 [  815.   803.   793. ...   751.   830.   903.]
 ...
 [24173. 24756. 24874. ... 21154. 21154. 20557.]
 [24756. 24874. 23944. ... 21154. 20557. 20853.]
 [24874. 23944. 22292. ... 20557. 20853. 22081.]]
(4896, 15)
              Q_-14    Q_-13    Q_-12    Q_-11    Q_-10     Q_-9     Q_-8  \
Date                                                                        
1985-05-01    824.0    814.0    815.0    803.0    793.0    800.0    805.0   
1985-05-02    814.0    815.0    803.0    793.0    800.0    805.0    813.0   
1985-05-03    815.0    803.0    793.0    800.0    805.0    813.0    824.0   
1985-05-04    803.0    793.0    800.0    805.0    813.0    824.0    829.0   
1985-05-05    793.0    800.0    805.0    813.0    824.0    829.0    829.0   
1985-05-06    800.0    805.0    813.0    824.0    829.0    829.0    852.0   
1985-05-07    805.0    813.0    824.0    829.0    829.0    852.0    810.0

# TODO:
MAKE EACH YEAR A BATCH!!!

In [15]:
RNN = initRNN()
RNN.compile(loss='mean_squared_error', optimizer = 'adam')
print(RNN.input_shape)
RNN.fit(X, y, epochs = numEpoch,  batch_size=1, 
         verbose=2, validation_split = .1)

(None, 1, 15)
Train on 4406 samples, validate on 490 samples
Epoch 1/5
 - 8s - loss: 695060179.7410 - val_loss: 459768013.7114
Epoch 2/5
 - 8s - loss: 693672543.4711 - val_loss: 458677493.7848
Epoch 3/5
 - 9s - loss: 692274064.1454 - val_loss: 457584966.2370
Epoch 4/5
 - 9s - loss: 690894364.6380 - val_loss: 456505394.1075
Epoch 5/5
 - 9s - loss: 689512834.2005 - val_loss: 455420315.2713


In [16]:
LSTM = initLSTM()
LSTM.compile(loss='mean_squared_error', optimizer = 'adam')
print(LSTM.input_shape)
LSTM.fit(X, y, epochs = numEpoch,  batch_size=1, 
         verbose=2, validation_split = .1)

(None, 1, 15)
Train on 4406 samples, validate on 490 samples
Epoch 1/5
 - 15s - loss: 695680252.1104 - val_loss: 460711242.7176
Epoch 2/5
 - 12s - loss: 695452544.6383 - val_loss: 460532857.8603
Epoch 3/5
 - 12s - loss: 695226628.1544 - val_loss: 460356309.0105
Epoch 4/5
 - 12s - loss: 694998626.4591 - val_loss: 460178416.8427
Epoch 5/5
 - 12s - loss: 694774150.5188 - val_loss: 460002304.3444


In [37]:
# Need to reshape dimensions for dense, see colab notebook

Dense = initDense()
Dense.compile(loss='mean_squared_error', optimizer = 'adam')
print(Dense.input_shape)
Dense.fit(X, y, epochs = numEpoch,  batch_size=1, 
         verbose=2, validation_split = .1)

(None, 1, 15)


ValueError: Error when checking target: expected dense_9 to have 3 dimensions, but got array with shape (4896, 1)